<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [4]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# '일시' 컬럼에서 '월', '일' 열 추가
data['월'] = data['일시'].dt.month
data['일'] = data['일시'].dt.day

# 월, 일별 중앙값 계산
monthly_medians = data.groupby('일').transform('median')
daily_medians = data.groupby('일').transform('median')

<ipython-input-4-f0d9f2a84bd0>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_medians = data.groupby('일').transform('median')
<ipython-input-4-f0d9f2a84bd0>:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily_medians = data.groupby('일').transform('median')


In [5]:
from datetime import datetime

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 29일에 해당하는 16개 행
# 23011 - 16 = 22995

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22992 non-null  float64       
 2   최저기온    22992 non-null  float64       
 3   일교차     22991 non-null  float64       
 4   강수량     9142 non-null   float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22991 non-null  float64       
 7   일조합     22877 non-null  float64       
 8   일사합     18137 non-null  float64       
 9   일조율     22630 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  월       22995 non-null  int64         
 12  일       22995 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(2)
memory usage: 2.5 MB
None


윤년 데이터 빼고 365로 나눠서 모델에 적용시키기

In [7]:
# 결측값 대체
data.fillna(daily_medians, inplace=True)

for column in data.columns:
    if data[column].isnull().any():  # 결측값이 있는 경우에만 처리
        data[column+'_중앙값'] = data.apply(lambda row: daily_medians.loc[row['일'], column], axis=1)
        data[column].fillna(data[column+'_중앙값'], inplace=True)
        data.drop(column+'_중앙값', axis=1, inplace=True)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22995 non-null  float64       
 2   최저기온    22995 non-null  float64       
 3   일교차     22995 non-null  float64       
 4   강수량     22995 non-null  float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22995 non-null  float64       
 7   일조합     22995 non-null  float64       
 8   일사합     22995 non-null  float64       
 9   일조율     22995 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  월       22995 non-null  int64         
 12  일       22995 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(2)
memory usage: 3.0 MB
None


In [8]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]

target = data['평균기온']

In [11]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(train, target)

pred_xgb = xgb.predict(train)

from sklearn.metrics import mean_squared_error
xgb_mae = mean_squared_error(target, pred_xgb)
print(xgb_mae)

submission['평균기온'] = pred_xgb[:358]
print(submission.head(10))

0.14606162682038204
           일시      평균기온
0  2023-01-01 -2.183585
1  2023-01-02 -2.091987
2  2023-01-03  3.852068
3  2023-01-04  6.974639
4  2023-01-05 -3.713041
5  2023-01-06 -5.435071
6  2023-01-07  1.221396
7  2023-01-08  1.920237
8  2023-01-09  2.150571
9  2023-01-10  1.890086


In [9]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train, target)

pred_rf = rf.predict(train)

from sklearn.metrics import mean_squared_error
rf_mae = mean_squared_error(target, pred_rf)
print(rf_mae)

submission['평균기온'] = pred_rf[:358]
print(submission.head(10))

0.03759200474016091
           일시   평균기온
0  2023-01-01 -1.645
1  2023-01-02 -2.074
2  2023-01-03  3.741
3  2023-01-04  6.982
4  2023-01-05 -4.270
5  2023-01-06 -5.253
6  2023-01-07  1.146
7  2023-01-08  1.814
8  2023-01-09  2.312
9  2023-01-10  1.974


다른 모델을 썼는데 어떻게 점수가 똑같이 나오지

예측에다가 위에 모델을 불러와서 그런거였고

In [10]:
submission.to_csv("DEAGYUL_12_submission.csv", index=False)

print(submission.head(30))

            일시    평균기온
0   2023-01-01  -1.645
1   2023-01-02  -2.074
2   2023-01-03   3.741
3   2023-01-04   6.982
4   2023-01-05  -4.270
5   2023-01-06  -5.253
6   2023-01-07   1.146
7   2023-01-08   1.814
8   2023-01-09   2.312
9   2023-01-10   1.974
10  2023-01-11  -1.963
11  2023-01-12  -3.706
12  2023-01-13  -3.218
13  2023-01-14  -1.724
14  2023-01-15  -5.022
15  2023-01-16  -4.969
16  2023-01-17  -5.466
17  2023-01-18  -2.831
18  2023-01-19   2.062
19  2023-01-20  -0.656
20  2023-01-21  -6.453
21  2023-01-22  -9.354
22  2023-01-23 -13.134
23  2023-01-24 -12.586
24  2023-01-25  -7.574
25  2023-01-26  -7.547
26  2023-01-27 -10.768
27  2023-01-28 -11.213
28  2023-01-29  -7.764
29  2023-01-30  -0.366


하다보니까 생각난건데 왜 평균값을 하면 안된다고 생각했지;; 0을 곱해버린다고 생각했다;;;